In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

In [2]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None
        
        new_row = pd.DataFrame({
                "account": ["WUSA"],
                "date": [time_now],
                "caption": str(event["description"])+" [For More Information, Click View Post] ",
                "display_photo": event["image"],
                "url": [event['url']],
                "likes": [0],
                "event_details": [{
                    "is_event": True,
                    "event_name": event["name"],
                    "event_description": str(event["description"])+" ... ",
                    "categories": [["SOCIAL"]],
                    "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()+ 7200),
                    "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()+ 7200),
                    "location": location,
                }]
        })

        postsDf = pd.concat([postsDf, new_row])

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

'streetAddress'
'location'
'location'
'streetAddress'
'location'
  account        date                                            caption  \
0    WUSA  1726096180  Don\'t miss out on the ultimate bowling experi...   
0    WUSA  1726096180  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1726096180  The Campus Life Fair is back!   Join us on Wed...   
0    WUSA  1726096180  Thrift Tea Experience the perfect blend of sty...   
0    WUSA  1726096180  📣 Looking to join a club and connect with like...   
0    WUSA  1726096180  The global movement known as &quot;Take Back t...   
0    WUSA  1726096180  📣 Looking to join a club and connect with like...   
0    WUSA  1726096180  In May, the Waterloo Undergraduate Student Ass...   
0    WUSA  1726096180  Embark on a flavour journey this fall and chan...   
0    WUSA  1726096180  In May, the Waterloo Undergraduate Student Ass...   
0    WUSA  1726096180  Embark on a flavour journey this fall and chan...   
0    WUSA  1726096180  

In [3]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with these IDs
result = collection.delete_many({'account': "WUSA"})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 7


In [4]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(0.5)


Inserted document ID: 66e2233ff6eba84ea2bb3177
Inserted document ID: 66e22340f6eba84ea2bb3178
Inserted document ID: 66e22342f6eba84ea2bb3179
Inserted document ID: 66e22343f6eba84ea2bb317a
Inserted document ID: 66e22345f6eba84ea2bb317b
Inserted document ID: 66e22346f6eba84ea2bb317c
Inserted document ID: 66e22347f6eba84ea2bb317d
Inserted document ID: 66e22348f6eba84ea2bb317e
Inserted document ID: 66e2234af6eba84ea2bb317f
Inserted document ID: 66e2234bf6eba84ea2bb3180
Inserted document ID: 66e2234df6eba84ea2bb3181
Inserted document ID: 66e2234df6eba84ea2bb3182
Inserted document ID: 66e2234ff6eba84ea2bb3183
Inserted document ID: 66e22350f6eba84ea2bb3184
Inserted document ID: 66e22352f6eba84ea2bb3185
Inserted document ID: 66e22353f6eba84ea2bb3186
Inserted document ID: 66e22354f6eba84ea2bb3187
